In [ ]:
ssh cc@129.114.108.8
sudo adduser daniar
sudo usermod -aG wheel daniar
sudo su 
cp -r /home/cc/.ssh /home/daniar
chmod 700  /home/daniar/.ssh
chmod 644  /home/daniar/.ssh/authorized_keys
chown daniar  /home/daniar/.ssh
chown daniar  /home/daniar/.ssh/authorized_keys
echo "daniar ALL=(ALL) NOPASSWD: ALL" >> /etc/sudoers.d/90-cloud-init-users
exit
exit

In [ ]:
# 0. Setup zsh [Login on "daniar"]

ssh daniar@129.114.108.8
sudo su
yum update -y
yum install zsh -y
chsh -s /bin/zsh root

In [ ]:
exit
sudo chsh -s /bin/zsh daniar
which zsh
echo $SHELL

sudo yum install wget git vim zsh -y

In [ ]:
printf 'Y' | sh -c "$(wget -O- https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh)"

/bin/cp ~/.oh-my-zsh/templates/zshrc.zsh-template ~/.zshrc
sudo sed -i 's|home/daniar:/bin/bash|home/daniar:/bin/zsh|g' /etc/passwd
sudo sed -i 's|ZSH_THEME="robbyrussell"|ZSH_THEME="risto"|g' ~/.zshrc
zsh
exit
exit

In [ ]:
# 2. Clone Motr and Install dependencies

sudo mkdir -p /mnt/extra
sudo chown daniar -R /mnt

cd /mnt/extra
git clone --recursive https://github.com/Seagate/cortx-motr.git

# install pip and python
yum group install -y "Development Tools"
yum install -y python-devel ansible tmux
curl https://bootstrap.pypa.io/pip/2.7/get-pip.py -o get-pip.py
python get-pip.py pip==19.3.1            
sudo pip install --target=/usr/lib64/python2.7/site-packages ipaddress

# force ansible to use python2
sudo su
echo "all:" >> /etc/ansible/hosts
echo "  ansible_python_interpreter: \"/usr/bin/python2\"" >> /etc/ansible/hosts
exit

# run build dependencies (9 min)
cd /mnt/extra/cortx-motr
sudo ./scripts/install-build-deps

In [ ]:
# 3. Configure libfabric 
    
cd /mnt/extra
wget https://github.com/Seagate/cortx/releases/download/build-dependencies/libfabric-1.11.2-1.el7.x86_64.rpm
wget https://github.com/Seagate/cortx/releases/download/build-dependencies/libfabric-devel-1.11.2-1.el7.x86_64.rpm
sudo rpm -i libfabric-1.11.2-1.el7.x86_64.rpm
sudo rpm -i libfabric-devel-1.11.2-1.el7.x86_64.rpm
sudo sed -i 's|tcp(eth1)|tcp(eth0)|g' /etc/libfab.conf

In [ ]:
# 4. Build motr 
# Make sure that the exampleX.c is in the ORIGINAL form, otherwise, you can't run "make"
# (7 min)
cd /mnt/extra/cortx-motr
sudo ./autogen.sh 

# Disable expensive checks 
    # https://cortxcommunity.slack.com/archives/C019S0SGWNQ/p1607974535256000
    
    sudo ./configure --disable-expensive-checks --disable-immediate-trace  --disable-dev-mode --with-trace-max-level=M0_ERROR
    
    # ./configure --with-trace-max-level=M0_INFO
    # ./configure --help | grep trace

cd /mnt/extra/cortx-motr
sudo make -j48

In [ ]:
# 5. Compile Python util
    
cd /mnt/extra/
sudo yum install -y gcc rpm-build python36 python36-pip python36-devel python36-setuptools openssl-devel libffi-devel python36-dbus
git clone --recursive https://github.com/Seagate/cortx-utils -b main

cd /mnt/extra/cortx-utils
./jenkins/build.sh -v 2.0.0 -b 2
sudo pip3 install -r https://raw.githubusercontent.com/Seagate/cortx-utils/main/py-utils/python_requirements.txt
sudo pip3 install -r https://raw.githubusercontent.com/Seagate/cortx-utils/main/py-utils/python_requirements.ext.txt

cd /mnt/extra/py-utils/dist
sudo yum install -y cortx-py-utils-*.noarch.rpm

In [ ]:
# 6. Building Hare 
    
cd /mnt/extra
# clone repo
git clone https://github.com/Seagate/cortx-hare.git && cd cortx-hare

# install fecter
sudo yum -y install python3 python3-devel yum-utils
sudo yum localinstall -y https://yum.puppetlabs.com/puppet/el/7/x86_64/puppet-agent-7.0.0-1.el7.x86_64.rpm
sudo ln -sf /opt/puppetlabs/bin/facter /usr/bin/facter

# install consul
sudo yum -y install yum-utils
sudo yum-config-manager --add-repo https://rpm.releases.hashicorp.com/RHEL/hashicorp.repo
sudo yum -y install consul-1.9.1

# build and install motr [here
cd /mnt/extra/cortx-motr && time sudo ./scripts/install-motr-service --link
export M0_SRC_DIR=$PWD

# build hare 
cd /mnt/extra/cortx-hare
sudo make
sudo make install

# create hare group
sudo groupadd --force hare
sudo usermod --append --groups hare $USER

# add path to zsh
echo 'PATH=/opt/seagate/cortx/hare/bin:$PATH' >> ~/.zshrc
echo 'export LD_LIBRARY_PATH=/mnt/extra/cortx-motr/motr/.libs/' >> ~/.zshrc
source ~/.zshrc